In [51]:
import os
import sys
import urllib
from urllib.request import urlopen
import numpy as np
import pandas as pd
import requests
from tqdm.notebook import trange, tqdm
import typing
import wget
import zipfile
from datetime import date, timedelta, datetime
import sys
import csv
# Incorporated to load the proper sized 
maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [4]:
os.getcwd()

'c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull'

## Getting the Master File Downloaded

In [5]:
# url variable - master list of all data files collected and stored by GDELT Project
url="http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"

In [6]:
text = requests.get(url).text.splitlines()

In [5]:
text[0]

'150383 297a16b493de7cf6ca809a7cc31d0b93 http://data.gdeltproject.org/gdeltv2/20150218230000.export.CSV.zip'

In [7]:
split_text = []
for i in tqdm(range(len(text))):
    split_line = text[i].split(' ')
    split_text.append(split_line)


  0%|          | 0/756951 [00:00<?, ?it/s]

- Woah there.  There are 756k rows of scraped information starting back in 2015 ad continuing up to today.  Is this all of the legendary 2.5TB they mentioned that they have?
- If this does in fact have all or even most of the data, there is no bloody way that I would be able to download and store it all.
- Let us start with a smaller set, say only looking at all the news over the last year or so.
- Once I have the full system in place, I can start to write all the scripts for handling the downloading and storing of the data for our usage (either personal or for Valkyrie's use)

In [8]:
df = pd.DataFrame(split_text)
df.head()

,0,1,2
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...


In [8]:
mask = df[2].str.contains('gdeltv2/20220601', na=False)
mask.value_counts()

False    756528
True        288
Name: 2, dtype: int64

In [9]:
df[mask]

,0,1,2
749544,152466,a6838081ea074f1853b32d26889f0379,http://data.gdeltproject.org/gdeltv2/202206010...
749545,110665,dc6dd46a024e9b07ee1a9d15f98de4c7,http://data.gdeltproject.org/gdeltv2/202206010...
749546,5818349,cbdf2f2eb2917986b96f7b455d3ef0e5,http://data.gdeltproject.org/gdeltv2/202206010...
749547,139778,611c45161cdf3fc26b1ccef0dea3774b,http://data.gdeltproject.org/gdeltv2/202206010...
749548,126558,fe867de1b332c589e677133e802545c9,http://data.gdeltproject.org/gdeltv2/202206010...
...,...,...,...
749827,148106,2923b9e6c38631a653c93e8cc9cbcca8,http://data.gdeltproject.org/gdeltv2/202206012...
749828,6912481,c092bde002ee42dfcb45ba378f67446e,http://data.gdeltproject.org/gdeltv2/202206012...
749829,86093,0ee7698a5f539c65e7720ba070a2185f,http://data.gdeltproject.org/gdeltv2/202206012...
749830,183502,a5c6b05d09e1562f90cdc33f42d13a8c,http://data.gdeltproject.org/gdeltv2/202206012...


- It appears the Death Star is orbiting the Forest Moon of Endor
- There were some `NaN` values in there that had to be handled apparently, but luckily the fine folks at pandas had a solution built in already.  Not sure how many data files I lose doing it this way.  Who knows.
- If I can get to a place where I can download a selected date range or a single day and process and store all the data, I think that I will be in good shape.
- There should be 288 entries for every day
    - This accounts for the 15 minute update
    - And for the three different files that are created and uploaded by GDELT Project.

In [10]:
mask_exports = (df[2].str.contains('gdeltv2/20220601', na=False) & df[2].str.contains('export', na=False))
df[mask_exports]

,0,1,2
749544,152466,a6838081ea074f1853b32d26889f0379,http://data.gdeltproject.org/gdeltv2/202206010...
749547,139778,611c45161cdf3fc26b1ccef0dea3774b,http://data.gdeltproject.org/gdeltv2/202206010...
749550,116579,deab31d4da2d9c0fab7f320e91047020,http://data.gdeltproject.org/gdeltv2/202206010...
749553,85662,1ccd0d1bb89c10a82370adf96391b345,http://data.gdeltproject.org/gdeltv2/202206010...
749556,103977,6aff19407893185483b47b721c7f256a,http://data.gdeltproject.org/gdeltv2/202206010...
...,...,...,...
749817,66114,43a39462bb330d626ebe2746f569c655,http://data.gdeltproject.org/gdeltv2/202206012...
749820,71673,73e47f93a243abf6f8e1e60a26f45725,http://data.gdeltproject.org/gdeltv2/202206012...
749823,67024,bced109c1a8067979e078c676e42a620,http://data.gdeltproject.org/gdeltv2/202206012...
749826,79330,46bb82b2d64959aea04103cf7d90fd64,http://data.gdeltproject.org/gdeltv2/202206012...


## Download Methods
- Download all files in the master archive file list
- Download a specific day
- Download over specific range
- If updating a local repository, then grab all files from last date in local repo to latest date available online

In [11]:
os.mkdir("data/")

In [13]:
os.listdir()

['data', 'gdelt-download.py']

In [17]:
df[2][0].split('/')

['http:',
 '',
 'data.gdeltproject.org',
 'gdeltv2',
 '20150218230000.export.CSV.zip']

In [24]:
a_date = '20220601'
main_path = "c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull"
os.getcwd()

'c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull'

In [32]:
a_date = '20220601'
a_type = 'export'
data_path = os.path.join(main_path, f"{a_date}")
print(data_path)
my_string = 'filepath'
os.path.join(data_path, my_string)

c:\Users\stark\Documents\Anastria\gdelt-data-pull\20220601


'c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull\\20220601\\filepath'

In [36]:
data_path = os.path.join(main_path, f"data\{a_date}")
print(data_path)

c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220601


In [47]:
file_string = '20220601020000.export.CSV.zip'
"-".join(file_string.split(".")[:2])

'20220601020000-export'

## Functions for Downloading

In [57]:
def download_single_day(date: str, master_archive: pd.DataFrame, type: str, dir_path: str):
    date_string = "gdeltv2/" + date
    mask = (master_archive[2].str.contains(date_string, na=False) & master_archive[2].str.contains(type, na=False))
    single_day = master_archive[mask]
    urls = single_day[2].to_list()
    date_path = os.path.join(dir_path, f"data\{date}")
    if not os.path.exists(date_path):
        os.makedirs(date_path)
    type_single_day_path = os.path.join(date_path, f"{type}")
    if not os.path.exists(type_single_day_path):
        os.makedirs(type_single_day_path)
    out_path = type_single_day_path
    for i in tqdm(range(len(urls)), desc=f"{date} {type} download progress"):
        wget.download(urls[i], type_single_day_path)
    print(f"Download of {date} {type} data complete")
    return out_path

main_path = "c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull"
download_single_day('20220530', df, 'export', main_path)

20220530 export download progress:   0%|          | 0/96 [00:00<?, ?it/s]

Download of 20220530 export data complete


'c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull\\data\\20220530\\export'

In [12]:
tmp_data_dir = "c://Users//stark//Documents//Anastria//gdelt-data-pull//data//20220601//export//"
os.listdir(tmp_data_dir)

['20220601000000.export.CSV.zip',
 '20220601001500.export.CSV.zip',
 '20220601003000.export.CSV.zip',
 '20220601004500.export.CSV.zip',
 '20220601010000.export.CSV.zip',
 '20220601011500.export.CSV.zip',
 '20220601013000.export.CSV.zip',
 '20220601014500.export.CSV.zip',
 '20220601020000.export.CSV.zip',
 '20220601021500.export.CSV.zip',
 '20220601023000.export.CSV.zip',
 '20220601024500.export.CSV.zip',
 '20220601030000.export.CSV.zip',
 '20220601031500.export.CSV.zip',
 '20220601033000.export.CSV.zip',
 '20220601034500.export.CSV.zip',
 '20220601040000.export.CSV.zip',
 '20220601041500.export.CSV.zip',
 '20220601043000.export.CSV.zip',
 '20220601044500.export.CSV.zip',
 '20220601050000.export.CSV.zip',
 '20220601051500.export.CSV.zip',
 '20220601053000.export.CSV.zip',
 '20220601054500.export.CSV.zip',
 '20220601060000.export.CSV.zip',
 '20220601061500.export.CSV.zip',
 '20220601063000.export.CSV.zip',
 '20220601064500.export.CSV.zip',
 '20220601070000.export.CSV.zip',
 '202206010715

In [48]:
def extract_zip(path: str):
    
    _EXPORT_COLUMN_NAMES = [
                            'GlobalEventID',
                            'Day',
                            'MonthYear',
                            'Year',
                            'FractionDate',
                            'Actor1Code',
                            'Actor1Name',
                            'Actor1CountryCode',
                            'Actor1KnownGroupCode',
                            'Actor1EthnicCode',
                            'Actor1Religion1Code',
                            'Actor1Religion2Code',
                            'Actor1Type1Code',
                            'Actor1Type2Code',
                            'Actor1Type3Code',
                            'Actor2Code',
                            'Actor2Name',
                            'Actor2CountryCode',
                            'Actor2KnownGroupCode',
                            'Actor2EthnicCode',
                            'Actor2Religion1Code',
                            'Actor2Religion2Code',
                            'Actor2Type1Code',
                            'Actor2Type2Code',
                            'Actor2Type3Code',
                            'IsRootEvent',
                            'EventCode',
                            'EventBaseCode',
                            'EventRotCode',
                            'QuadClass',
                            'GoldsteinScale',
                            'NumMenions',
                            'NumSources',
                            'NumArticles',
                            'AvgTone',
                            'Actor1Geo_Type',
                            'Actor1Geo_FullName',
                            'Actor1Geo_CountryCode',
                            'Actor1Geo_ADM1Code',
                            'Actor1Geo_ADM2Code',
                            'Actor1Geo_Lat',
                            'Actor1Geo_Lon',
                            'Actor1Geo_FeatureID',
                            'Actor2Geo_Type',
                            'Actor2Geo_FullName',
                            'Actor2Geo_CountryCode',
                            'Actor2Geo_ADM1Code',
                            'Actor2Geo_ADM2Code',
                            'Actor2Geo_Lat',
                            'Actor2Geo_Lon',
                            'Actor2Geo_FeatureID',
                            'DATEADDED',
                            'SOURCEURL'
                           ]
    _MENTIONS_COLUMN_NAMES = [
                              'GlobalEventID',
                              'EventTimeDate',
                              'MentionTimeDate',
                              'MentionType',
                              'MentionSourceName',
                              'MentionIdentifer',
                              'SentenceID',
                              'Actor1CharOffset',
                              'Actor2CharOffset',
                              'ActionCharOffset',
                              'InRawText',
                              'Confidence',
                              'MentionDocLen',
                              'MentionDocTone',
                              'MentionTranslationInfo',
                              'Extras'
                             ]
    _GKG_COLUMN_NAMES = [
                            'GlobalEventID',
                            'Day',
                            'MonthYear',
                            'Year',
                            'FractionDate',
                            'Actor1Code',
                            'Actor1Name',
                            'Actor1CountryCode',
                            'Actor1KnownGroupCode',
                            'Actor1EthnicCode',
                            'Actor1Religion1Code',
                            'Actor1Religion2Code',
                            'Actor1Type1Code',
                            'Actor1Type2Code',
                            'Actor1Type3Code',
                            'Actor2Code',
                            'Actor2Name',
                            'Actor2CountryCode',
                            'Actor2KnownGroupCode',
                            'Actor2EthnicCode',
                            'Actor2Religion1Code',
                            'Actor2Religion2Code',
                            'Actor2Type1Code',
                            'Actor2Type2Code',
                            'Actor2Type3Code',
                            'IsRootEvent',
                            'EventCode',
                            'EventBaseCode',
                            'EventRotCode',
                            'QuadClass',
                            'GoldsteinScale',
                            'NumMenions',
                            'NumSources',
                            'NumArticles',
                            'AvgTone',
                            'Actor1Geo_Type',
                            'Actor1Geo_FullName',
                            'Actor1Geo_CountryCode',
                            'Actor1Geo_ADM1Code',
                            'Actor1Geo_ADM2Code',
                            'Actor1Geo_Lat',
                            'Actor1Geo_Lon',
                            'Actor1Geo_FeatureID',
                            'Actor2Geo_Type',
                            'Actor2Geo_FullName',
                            'Actor2Geo_CountryCode',
                            'Actor2Geo_ADM1Code',
                            'Actor2Geo_ADM2Code',
                            'Actor2Geo_Lat',
                            'Actor2Geo_Lon',
                            'Actor2Geo_FeatureID',
                            'DATEADDED',
                            'SOURCEURL'
    ]
    files = os.listdir(path)
    print(f"Unzipping and saving files as parquet in {path}")
    for i in tqdm(range(len(files)), desc='Unzipping/Saving Progress'):
        file = files[i]
        file_path = os.path.join(path, file)
        save_name = "-".join(file.split(".")[:2]) + ".parquet"
        extracted_data = pd.read_csv(file_path,
                                     compression="zip",
                                     header=None,
                                     delimiter="\t",
                                     engine='python',
                                     encoding_errors='ignore'
                                    )
        extracted_data.columns = [str(x) for x in extracted_data.columns.to_list()]
        extracted_data.to_parquet(os.path.join(path, save_name))
    print("Unziping and saving completed.")
    print(f"Removing zip files from {path}")
    for i in tqdm(range(len(files)), desc="Removing zipfiles progress"):
        file_to_be_removed_path = os.path.join(path, files[i])
        os.remove(file_to_be_removed_path)
    print(f"Zipped files removed in {path}")

#extract_zip(tmp_data_dir)

In [54]:
[str(x) for x in df.columns.to_list()]

['0', '1', '2']

In [40]:
from datetime import date, timedelta
a_date = '20220601'
conv_date = datetime.strptime(a_date, "%Y%m%d").date()
start_date = conv_date
end_date = date(2022, 6, 3)    # perhaps date.now()

delta = end_date - start_date   # returns timedelta
print(conv_date)
print(type(conv_date))
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    day = day.strftime("%Y%m%d")
    print(day)

2022-06-01
<class 'datetime.date'>
20220601
20220602
20220603


In [52]:
def download_date_range(start_date: str, end_date: str, master_archive: pd.DataFrame, dir_path: str):
    
    start_dt = datetime.strptime(start_date, "%Y%m%d").date()
    end_dt = datetime.strptime(end_date, "%Y%m%d").date()
    t_delta = end_dt - start_dt
    days = []
    for i in range(delta.days + 1):
        day = start_dt + timedelta(days=i)
        day = day.strftime("%Y%m%d")
        days.append(day)
    print(days)
    for day in days:
        day_export_path = download_single_day(day, master_archive, 'export', dir_path)
        extract_zip(path=day_export_path)
        day_mentions_path = download_single_day(day, master_archive, 'mentions', dir_path)
        extract_zip(path=day_mentions_path)
        day_gkg_path = download_single_day(day, master_archive, 'gkg', dir_path)
        extract_zip(path=day_gkg_path)


main_path = "c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull"
download_date_range(start_date='20220601',
                    end_date='20220602',
                    master_archive=df,
                    dir_path=main_path)

['20220601', '20220602', '20220603']


20220601 export download progress:   0%|          | 0/96 [00:00<?, ?it/s]

Download of 20220601 export data complete
Unzipping and saving files as parquet in c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220601\export


Unzipping/Saving Progress:   0%|          | 0/96 [00:00<?, ?it/s]

Unziping and saving completed.
Removing zip files from c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220601\export


...

Download of 20220603 gkg data complete
Unzipping and saving files as parquet in c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220603\gkg


Unzipping/Saving Progress:   0%|          | 0/96 [00:00<?, ?it/s]

Unziping and saving completed.
Removing zip files from c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220603\gkg


Removing zipfiles progress:   0%|          | 0/96 [00:00<?, ?it/s]

Zipped files removed in c:\Users\stark\Documents\Anastria\gdelt-data-pull\data\20220603\gkg


In [53]:
os.getcwd()

'c:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull'

In [56]:
pd.set_option('display.max_columns', None)
test_file = r"C:\\Users\\stark\\Documents\\Anastria\\gdelt-data-pull\\data\\20220601\\export\\20220601000000-export.parquet"
tmp_df = pd.read_parquet(test_file)
tmp_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,1047200008,20210601,202106,2021,2021.4137,IGOUNO,UNITED NATIONS,None,UNO,None,None,None,IGO,None,None,None,None,None,None,None,None,None,None,None,NaN,0,10,10,1,1,0.0,10,2,10,3.030303,3,"White House, District of Columbia, United States",US,USDC,None,38.895100,-77.0364,531871,0,None,None,None,None,NaN,NaN,None,3,"White House, District of Columbia, United States",US,USDC,None,38.895100,-77.0364,531871,20220601000000,https://943thepoint.com/bts-full-white-house-s...
1,1047200009,20210601,202106,2021,2021.4137,JEW,JEWISH,None,None,None,JEW,None,None,None,None,NMR,NORTH AMERICA,NMR,None,None,None,None,None,None,NaN,0,71,71,7,2,7.4,1,1,1,0.377358,4,"Jerusalem, Israel (general), Israel",IS,IS00,18315,31.766700,35.2333,-797092,4,"Jerusalem, Israel (general), Israel",IS,IS00,18315,31.7667,35.2333,-797092,4,"Gondar, Amara, Ethiopia",ET,ET46,149280,12.600000,37.4667,-612097,20220601000000,https://www.clevelandjewishnews.com/jta/after-...
2,1047200010,20210601,202106,2021,2021.4137,MED,MEDIA,None,None,None,None,None,MED,None,None,JUD,FEDERAL JUDGE,None,None,None,None,None,JUD,None,NaN,0,190,190,19,4,-10.0,10,1,10,-3.501401,2,"Florida, United States",US,USFL,None,27.833300,-81.7170,FL,2,"Florida, United States",US,USFL,None,27.8333,-81.7170,FL,2,"Florida, United States",US,USFL,None,27.833300,-81.7170,FL,20220601000000,https://www.houstonchronicle.com/politics/texa...
3,1047200011,20210601,202106,2021,2021.4137,SGP,SINGAPORE,SGP,None,None,None,None,None,None,None,MYS,MALAYSIA,MYS,None,None,None,None,None,None,NaN,0,61,61,6,2,6.4,2,1,2,-3.647416,4,"Johor, Johor, Malaysia",MY,MY01,18585,1.465500,103.7580,-2405456,4,"Johor, Johor, Malaysia",MY,MY01,18585,1.4655,103.7580,-2405456,1,Singapore,SN,SN,None,1.366667,103.8000,SN,20220601000000,https://www.efeedlink.com/contents/06-01-2022/...
4,1047200012,20210601,202106,2021,2021.4137,SGP,SINGAPORE,SGP,None,None,None,None,None,None,None,MYS,MALAYSIA,MYS,None,None,None,None,None,None,NaN,0,61,61,6,2,6.4,8,1,8,-3.647416,1,Singapore,SN,SN,None,1.366667,103.8000,SN,4,"Johor, Johor, Malaysia",MY,MY01,18585,1.4655,103.7580,-2405456,1,Singapore,SN,SN,None,1.366667,103.8000,SN,20220601000000,https://www.efeedlink.com/contents/06-01-2022/...
